# Entendiendo WeightedFamiliesLayerPytorch

Este notebook explica paso a paso cómo funciona la clase `WeightedFamiliesLayerPytorch` con valores determinísticos.

In [15]:
import torch as tr
import torch.nn as nn

## 1. La clase completa

In [16]:
class WeightedFamiliesLayerPytorch(nn.Module):
    def __init__(self, num_models, num_classes, hidden_size=4):
        super().__init__()

        # First layer: per-model, per-family
        self.weights1 = nn.Parameter(tr.rand(num_models, num_classes, hidden_size))
        self.bias1 = nn.Parameter(tr.zeros(num_classes, hidden_size))

        # Second layer: per-family
        self.weights2 = nn.Parameter(tr.rand(num_classes, hidden_size))
        self.bias2 = nn.Parameter(tr.zeros(num_classes))

    def forward(self, x): # x: (num_models, batch, num_classes)
        # Expand x to (num_models, batch, num_classes, 1)
        x = x.unsqueeze(-1)

        # First aggregation (models → hidden)
        w1 = self.weights1.unsqueeze(1)  # (num_models, 1, num_classes, hidden_size)
        b1 = self.bias1.unsqueeze(0) # (1, num_classes, hidden_size)
        h = tr.sum(x * w1, dim=0) + b1
        h = tr.relu(h)  # (batch, num_classes, hidden_size)

        # Second aggregation (hidden → output)
        out = tr.sum(h * self.weights2.unsqueeze(0), dim=-1) + self.bias2
        return out  # (batch, num_classes)

## 2. Configuración de ejemplo

Usaremos valores pequeños y determinísticos para entender cada paso:
- **num_models = 2** (dos modelos en el ensemble)
- **num_classes = 3** (tres familias de proteínas)
- **batch = 2** (dos ejemplos en el batch)
- **hidden_size = 2** (dimensión oculta pequeña)

In [17]:
num_models = 2
num_classes = 3
batch_size = 2
hidden_size = 2

# Crear la capa
layer = WeightedFamiliesLayerPytorch(num_models, num_classes, hidden_size)

print("Parámetros de la capa:")
print(f"weights1 shape: {layer.weights1.shape}")
print(f"bias1 shape: {layer.bias1.shape}")
print(f"weights2 shape: {layer.weights2.shape}")
print(f"bias2 shape: {layer.bias2.shape}")

Parámetros de la capa:
weights1 shape: torch.Size([2, 3, 2])
bias1 shape: torch.Size([3, 2])
weights2 shape: torch.Size([3, 2])
bias2 shape: torch.Size([3])


## 3. Setear valores determinísticos

Para entender mejor, asignamos valores fijos en lugar de aleatorios.

In [4]:
# weights1: (num_models=2, num_classes=3, hidden_size=2)
layer.weights1.data = tr.tensor([
    [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]],  # Modelo 1
    [[0.5, 1.0], [1.5, 2.0], [2.5, 3.0]]   # Modelo 2
], dtype=tr.float32)

# bias1: (num_classes=3, hidden_size=2)
layer.bias1.data = tr.tensor([
    [0.1, 0.2],  # Clase 0
    [0.3, 0.4],  # Clase 1
    [0.5, 0.6]   # Clase 2
], dtype=tr.float32)

# weights2: (num_classes=3, hidden_size=2)
layer.weights2.data = tr.tensor([
    [1.0, 1.0],  # Clase 0
    [2.0, 2.0],  # Clase 1
    [3.0, 3.0]   # Clase 2
], dtype=tr.float32)

# bias2: (num_classes=3)
layer.bias2.data = tr.tensor([0.1, 0.2, 0.3], dtype=tr.float32)

print("Pesos seteados exitosamente")

Pesos seteados exitosamente


## 4. Input de ejemplo

Creamos un input `x` con forma `(num_models, batch, num_classes)` con valores determinísticos.

In [19]:
# x: (num_models=2, batch=2, num_classes=3)
x = tr.tensor([
    [[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]],  # Modelo 1, batch 0 y 1
    [[0.7, 0.8, 0.9], [1.0, 1.1, 1.2]]   # Modelo 2, batch 0 y 1
], dtype=tr.float32)

print("Input x:")
print(x)
print(f"Shape: {x.shape}")

Input x:
tensor([[[0.1000, 0.2000, 0.3000],
         [0.4000, 0.5000, 0.6000]],

        [[0.7000, 0.8000, 0.9000],
         [1.0000, 1.1000, 1.2000]]])
Shape: torch.Size([2, 2, 3])


## 5. Paso 1: unsqueeze(-1)

Añadimos una dimensión extra al final para poder multiplicar con los pesos.

In [20]:
x_expanded = x.unsqueeze(-1)
print("x después de unsqueeze(-1):")
print(x_expanded)
print(f"Shape: {x_expanded.shape}")
print(f"\nCambió de {x.shape} a {x_expanded.shape}")

x después de unsqueeze(-1):
tensor([[[[0.1000],
          [0.2000],
          [0.3000]],

         [[0.4000],
          [0.5000],
          [0.6000]]],


        [[[0.7000],
          [0.8000],
          [0.9000]],

         [[1.0000],
          [1.1000],
          [1.2000]]]])
Shape: torch.Size([2, 2, 3, 1])

Cambió de torch.Size([2, 2, 3]) a torch.Size([2, 2, 3, 1])


## 6. Paso 2: Preparar weights1 para broadcasting

Expandimos `weights1` para que tenga una dimensión de batch.

In [21]:
w1 = layer.weights1.unsqueeze(1)
print("weights1 original shape:", layer.weights1.shape)
print("w1 después de unsqueeze(1):")
print(w1)
print(f"Shape: {w1.shape}")

weights1 original shape: torch.Size([2, 3, 2])
w1 después de unsqueeze(1):
tensor([[[[0.4602, 0.4135],
          [0.8105, 0.1076],
          [0.2113, 0.5382]]],


        [[[0.3064, 0.7573],
          [0.3055, 0.7222],
          [0.1363, 0.6831]]]], grad_fn=<UnsqueezeBackward0>)
Shape: torch.Size([2, 1, 3, 2])


## 7. Paso 3: Multiplicación $x * w1$ (primera agregación)

Broadcasting permite multiplicar:
- `x_expanded`: `(2, 2, 3, 1)`
- `w1`: `(2, 1, 3, 2)`

Resultado: `(2, 2, 3, 2)` (se expanden las dimensiones con 1)

In [22]:
multiplied = x_expanded * w1
print("x_expanded * w1:")
print(multiplied)
print(f"Shape: {multiplied.shape}")

print("\nEjemplo para modelo=0, batch=0, clase=0:")
print(f"x[0,0,0] = {x[0,0,0]:.2f}")
print(f"w1[0,0,0,:] = {w1[0,0,0,:]}")
print(f"Resultado: {multiplied[0,0,0,:]}")

x_expanded * w1:
tensor([[[[0.0460, 0.0414],
          [0.1621, 0.0215],
          [0.0634, 0.1615]],

         [[0.1841, 0.1654],
          [0.4053, 0.0538],
          [0.1268, 0.3229]]],


        [[[0.2145, 0.5301],
          [0.2444, 0.5778],
          [0.1226, 0.6148]],

         [[0.3064, 0.7573],
          [0.3361, 0.7944],
          [0.1635, 0.8197]]]], grad_fn=<MulBackward0>)
Shape: torch.Size([2, 2, 3, 2])

Ejemplo para modelo=0, batch=0, clase=0:
x[0,0,0] = 0.10
w1[0,0,0,:] = tensor([0.4602, 0.4135], grad_fn=<SliceBackward0>)
Resultado: tensor([0.0460, 0.0414], grad_fn=<SliceBackward0>)


## 8. Paso 4: Sumar sobre dim=0 (modelos)

Colapsamos la dimensión de modelos, combinando las predicciones de ambos modelos.

In [9]:
summed = tr.sum(multiplied, dim=0)
print("Suma sobre dim=0 (modelos):")
print(summed)
print(f"Shape: {summed.shape}")

print("\nVerificación manual para batch=0, clase=0:")
print(f"Modelo1: x[0,0,0] * w1[0,0,0,:] = {x[0,0,0]:.2f} * {w1[0,0,0,:]} = {x[0,0,0] * w1[0,0,0,:]}")
print(f"Modelo2: x[1,0,0] * w1[1,0,0,:] = {x[1,0,0]:.2f} * {w1[1,0,0,:]} = {x[1,0,0] * w1[1,0,0,:]}")
print(f"Suma: {summed[0,0,:]}")

Suma sobre dim=0 (modelos):
tensor([[[0.4500, 0.9000],
         [1.8000, 2.4000],
         [3.7500, 4.5000]],

        [[0.9000, 1.8000],
         [3.1500, 4.2000],
         [6.0000, 7.2000]]], grad_fn=<SumBackward1>)
Shape: torch.Size([2, 3, 2])

Verificación manual para batch=0, clase=0:
Modelo1: x[0,0,0] * w1[0,0,0,:] = 0.10 * tensor([1., 2.], grad_fn=<SliceBackward0>) = tensor([0.1000, 0.2000], grad_fn=<MulBackward0>)
Modelo2: x[1,0,0] * w1[1,0,0,:] = 0.70 * tensor([0.5000, 1.0000], grad_fn=<SliceBackward0>) = tensor([0.3500, 0.7000], grad_fn=<MulBackward0>)
Suma: tensor([0.4500, 0.9000], grad_fn=<SliceBackward0>)


## 9. Paso 5: Añadir bias1

El bias tiene forma `(num_classes, hidden_size)`, lo expandimos para batch.

In [10]:
b1 = layer.bias1.unsqueeze(0)
print("bias1 original:", layer.bias1.shape)
print("b1 después de unsqueeze(0):", b1.shape)
print(b1)

h_before_relu = summed + b1
print("\nSuma + bias1:")
print(h_before_relu)
print(f"Shape: {h_before_relu.shape}")

bias1 original: torch.Size([3, 2])
b1 después de unsqueeze(0): torch.Size([1, 3, 2])
tensor([[[0.1000, 0.2000],
         [0.3000, 0.4000],
         [0.5000, 0.6000]]], grad_fn=<UnsqueezeBackward0>)

Suma + bias1:
tensor([[[0.5500, 1.1000],
         [2.1000, 2.8000],
         [4.2500, 5.1000]],

        [[1.0000, 2.0000],
         [3.4500, 4.6000],
         [6.5000, 7.8000]]], grad_fn=<AddBackward0>)
Shape: torch.Size([2, 3, 2])


## 10. Paso 6: Aplicar ReLU

La activación ReLU convierte valores negativos a 0.

In [23]:
h = tr.relu(h_before_relu)
print("Después de ReLU:")
print(h)
print(f"Shape: {h.shape}")

print("\nNota: ReLU(x) = max(0, x), así que los valores negativos se vuelven 0")

Después de ReLU:
tensor([[[0.5500, 1.1000],
         [2.1000, 2.8000],
         [4.2500, 5.1000]],

        [[1.0000, 2.0000],
         [3.4500, 4.6000],
         [6.5000, 7.8000]]], grad_fn=<ReluBackward0>)
Shape: torch.Size([2, 3, 2])

Nota: ReLU(x) = max(0, x), así que los valores negativos se vuelven 0


## 11. Paso 7: Segunda agregación (hidden → output)

Multiplicamos `h` por `weights2` y sumamos sobre la dimensión oculta.

In [12]:
w2 = layer.weights2.unsqueeze(0)
print("weights2 original:", layer.weights2.shape)
print("w2 después de unsqueeze(0):", w2.shape)
print(w2)

# Multiplicación
h_w2 = h * w2
print("\nh * w2:")
print(h_w2)
print(f"Shape: {h_w2.shape}")

# Suma sobre hidden_size (dim=-1)
out_before_bias = tr.sum(h_w2, dim=-1)
print("\nSuma sobre dim=-1 (hidden_size):")
print(out_before_bias)
print(f"Shape: {out_before_bias.shape}")

weights2 original: torch.Size([3, 2])
w2 después de unsqueeze(0): torch.Size([1, 3, 2])
tensor([[[1., 1.],
         [2., 2.],
         [3., 3.]]], grad_fn=<UnsqueezeBackward0>)

h * w2:
tensor([[[ 0.5500,  1.1000],
         [ 4.2000,  5.6000],
         [12.7500, 15.3000]],

        [[ 1.0000,  2.0000],
         [ 6.9000,  9.2000],
         [19.5000, 23.4000]]], grad_fn=<MulBackward0>)
Shape: torch.Size([2, 3, 2])

Suma sobre dim=-1 (hidden_size):
tensor([[ 1.6500,  9.8000, 28.0500],
        [ 3.0000, 16.1000, 42.9000]], grad_fn=<SumBackward1>)
Shape: torch.Size([2, 3])


## 12. Paso 8: Añadir bias2 (output final)

In [13]:
out = out_before_bias + layer.bias2
print("Output final:")
print(out)
print(f"Shape: {out.shape}")

print("\nbias2:", layer.bias2)

Output final:
tensor([[ 1.7500, 10.0000, 28.3500],
        [ 3.1000, 16.3000, 43.2000]], grad_fn=<AddBackward0>)
Shape: torch.Size([2, 3])

bias2: Parameter containing:
tensor([0.1000, 0.2000, 0.3000], requires_grad=True)


## 13. Verificación: Comparar con la capa completa

In [14]:
# Calcular con la capa completa
output_layer = layer(x)

print("Output de la capa completa:")
print(output_layer)

print("\nOutput calculado manualmente:")
print(out)

print("\n¿Son iguales?")
print(tr.allclose(output_layer, out))

Output de la capa completa:
tensor([[ 1.7500, 10.0000, 28.3500],
        [ 3.1000, 16.3000, 43.2000]], grad_fn=<AddBackward0>)

Output calculado manualmente:
tensor([[ 1.7500, 10.0000, 28.3500],
        [ 3.1000, 16.3000, 43.2000]], grad_fn=<AddBackward0>)

¿Son iguales?
True


## 14. Resumen visual

```
Input: (num_models, batch, num_classes)
   ↓ unsqueeze(-1)
(num_models, batch, num_classes, 1)
   ↓ × weights1 (broadcasted)
(num_models, batch, num_classes, hidden_size)
   ↓ sum(dim=0) + bias1
(batch, num_classes, hidden_size)
   ↓ ReLU
(batch, num_classes, hidden_size)
   ↓ × weights2 + sum(dim=-1) + bias2
(batch, num_classes) ← Output final
```

## 15. ¿Qué hace esta capa?

1. **Primera capa**: Aprende una combinación ponderada de las predicciones de cada modelo para cada familia, proyectándolas a un espacio oculto (hidden_size). Esto permite capturar interacciones complejas.

2. **ReLU**: Introduce no-linealidad, permitiendo que la red aprenda patrones más complejos que una simple suma ponderada.

3. **Segunda capa**: Colapsa el espacio oculto de vuelta a una predicción por familia.

**Diferencia con WeightedFamiliesLayer**: Esta versión tiene una capa oculta que permite aprender relaciones no-lineales entre los modelos, mientras que la original solo hace una suma ponderada lineal.